In [2]:
import pandas as pd
import numpy as np

#yelp_business = pd.read_json('yelp_academic_dataset_business.json')


# read the entire file into a python array
with open('yelp_academic_dataset_business.json', 'rb') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
data_json_str = b'[' + b','.join(data) + b']'

# now, load it into pandas
data_df = pd.read_json(data_json_str)
data_df.head(2)

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,"{u'Take-out': True, u'Drive-Thru': False, u'Ou...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",Dravosburg,"4734 Lebanon Church Rd\nDravosburg, PA 15034","{u'Tuesday': {u'close': u'21:00', u'open': u'1...",40.354327,-79.900706,Mr Hoagie,[],True,4,4.5,PA,business
1,"{u'Happy Hour': True, u'Accepts Credit Cards':...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],Dravosburg,"202 McClure St\nDravosburg, PA 15034",{},40.350553,-79.886814,Clancy's Pub,[],True,4,3.5,PA,business


In [3]:
from copy import deepcopy
attributes_bool_atts = set()
attributes_categor_atts = {} # the attribute maps to it's values
attributes_continuous_atts = {} # this only contains the integer price range from 1 to 4, by the way
attributes_dict_atts = {} # the attribute maps to a map which contains attributes to booleans
categories_atts = set() # boolean

rest_data_df = data_df[['Restaurants' in x for x in data_df['categories']]]
print(len(rest_data_df))
     
count = 0.0
index = 0.0
from collections import defaultdict
counts = defaultdict(int)

# find the features that exist; want to expand every feature to it's own column (and get rid of categoricals)
for restaurant in rest_data_df['attributes']:
    counts[len(restaurant)] += 1
    index += 1
    for attName in restaurant:
        attVal = restaurant[attName]
        T = type(attVal)
        if T is bool:
            attributes_bool_atts.add(attName)
        elif T is unicode:
            if attName not in attributes_categor_atts:
                attributes_categor_atts[attName] = set()
            attributes_categor_atts[attName].add(attVal)
        elif T is int:
            if attName not in attributes_continuous_atts:
                attributes_continuous_atts[attName] = set()
            attributes_continuous_atts[attName].add(attVal)
        elif T is dict:
            if attName not in attributes_dict_atts:
                attributes_dict_atts[attName] = set()
            for k in attVal:
                attributes_dict_atts[attName].add(k)

for restaurant in rest_data_df['categories']:
    for category in restaurant:
        categories_atts.add(category)
        


# now insert the columns into another dataframe
rest_expanded_df = deepcopy(rest_data_df)

for attrName in attributes_bool_atts:
    rest_expanded_df.loc[:,attrName] = [int(attr[attrName]) if attrName in attr else None for attr in rest_data_df['attributes']]

for outer in attributes_categor_atts:
    for attrName in attributes_categor_atts[outer]:
        rest_expanded_df.loc[:,attrName] = [int(attr[attrName]) if attrName in attr else None for attr in rest_data_df['attributes']]

for outer in attributes_continuous_atts:
    for attrName in attributes_continuous_atts[outer]:
        rest_expanded_df.loc[:,attrName] = [int(attr[attrName]) if attrName in attr else None for attr in rest_data_df['attributes']]

for outer in attributes_dict_atts:
    for attrName in attributes_dict_atts[outer]:
        #featureName = "{}_{}".format(outer,attrName)
        rest_expanded_df.loc[:,attrName] = [int(attr[outer][attrName]) if (outer in attr and attrName in attr[outer]) else None for attr in rest_data_df['attributes']]
        
for attrName in categories_atts:
    rest_expanded_df.loc[:,attrName] = [int(attrName in r) for r in rest_data_df['categories']]
    


regression_target = rest_expanded_df['stars'].values
median = np.median(regression_target)
classification_target = [1 if stars > median else 0 for stars in regression_target]
# now that we've expanded the categorical/boolean stuff
removalList = ['attributes', 'categories', 'city','full_address','hours','latitude','longitude','name','neighborhoods',
              'open','stars','state','type']
rest_lv_df = rest_expanded_df.loc[rest_expanded_df['city']==u'Las Vegas']

25071


In [37]:
# just wanted to see the top cities, looks like las vegas, followed by phoenix have the most restaurants
'''
cities = defaultdict(int)
for city in rest_expanded_df['city']:
    cities[city] += 1
print cities
'''  


#rest_lv_df.head(5)

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,...,"Books, Mags, Music & Video",Modern European,Arcades,Cambodian,Specialty Food,Spanish,Festivals,Cafeteria,Curry Sausage,Soul Food
7984,"{u'BYOB': True, u'Take-out': False, u'Alcohol'...",_SM8UKIwBNbmj1r629ipoQ,"[Wine Bars, Bars, Restaurants, Nightlife, Ital...",Las Vegas,"1916 Village Center Cir\nSummerlin\nLas Vegas,...","{u'Monday': {u'close': u'23:00', u'open': u'11...",36.193609,-115.304009,Chianti Cafe,[Summerlin],...,False,False,False,False,False,False,False,False,False,False
8226,"{u'Drive-Thru': False, u'Alcohol': u'none', u'...",l6QcUE8XXLrVH6Ydm4GSNw,"[Burgers, American (Traditional), Fast Food, R...",Las Vegas,Texas Station Casino\n2101 Texas Star Ln\nLas ...,"{u'Monday': {u'close': u'00:00', u'open': u'00...",36.197409,-115.191385,Fatburger,[],...,False,False,False,False,False,False,False,False,False,False
8290,"{u'Take-out': True, u'Alcohol': u'none', u'Del...",AgtFQVxo-PGQVsOdY6Nvdg,"[Pizza, Restaurants]",Las Vegas,"10525 Eastern Ave\nSte 130\nAnthem\nLas Vegas,...",{},35.999205,-115.107474,Pizza Hut,[Anthem],...,False,False,False,False,False,False,False,False,False,False
8297,"{u'Wheelchair Accessible': True, u'Take-out': ...",muqFM9Hoamh_fGL4MPeZqg,"[Pubs, Bars, Nightlife, Irish, Restaurants]",Las Vegas,"9470 Eastern Ave\nSoutheast\nLas Vegas, NV 89123","{u'Monday': {u'close': u'02:00', u'open': u'11...",36.017605,-115.117891,Fado Irish Pub,[Southeast],...,False,False,False,False,False,False,False,False,False,False
8488,"{u'Wheelchair Accessible': True, u'Take-out': ...",AFDFOs2GaIyUArGyEUDQzQ,"[Chinese, Restaurants]",Las Vegas,"4355 Spring Mountain Rd\nChinatown\nLas Vegas,...","{u'Monday': {u'close': u'02:00', u'open': u'11...",36.125908,-115.197904,Cafe Noodle & Chinese Barbeque,[Chinatown],...,False,False,False,False,False,False,False,False,False,False


In [4]:
import numpy as np

# targets for prediction
regression_target = rest_lv_df['stars'].values
median = np.median(regression_target)
classification_target = [1 if stars > median else 0 for stars in regression_target]

X = deepcopy(rest_lv_df)
y = classification_target

# now that we've expanded the categorical/boolean stuff, remove columns we won't predict on
removalList = ['attributes','business_id', 'categories', 'city','full_address','hours','latitude','longitude','name','neighborhoods',
              'open','stars','state','type']
for rm in removalList:
    X.drop(rm, axis=1, inplace=True)

#dummy_vars = []
#dummy_vars += list(attributes_bool_atts) + attributes_categor_atts.values() + attributes_continuous_atts.values() + attributes_dict_atts.values() + list(categories_atts)


In [5]:
import math
nans = defaultdict(int)
catch = defaultdict(int)

#df_norm = (X - X.mean()) / (X.max() - X.min())
#df_dummies.head(2)
X = X.fillna(0)
print X.shape
#X.head(3)

#print [i for i in df_dummies.columns if i not in X.columns]
#print df_dummies.columns - X.columns

(4658, 376)


In [6]:
from sklearn import svm
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

def getLabels(y_test):
    targetSet = set()
    for y in y_test:
        targetSet.add(y)
    return sorted(list(targetSet))

def evaluateClassifier(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion matrix\n', cm)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print('Normalized confusion matrix\n', cm_normalized)
    perClassAvg = np.mean(cm_normalized.diagonal())
    
    
    targets = getLabels(y_test)
    for indx, val in enumerate(cm):
        print("{} accuracy: {}".format(targets[indx], val[indx]/sum(val)))
    print("Per-class accuracy: {}".format(perClassAvg))
    print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
rbf = svm.SVC(kernel='rbf').fit(X_train, y_train)
y_pred = rbf.predict(X_test)
evaluateClassifier(y_test, y_pred)

print()


('Confusion matrix\n', array([[904,  86],
       [440, 108]]))
('Normalized confusion matrix\n', array([[ 0.91313131,  0.08686869],
       [ 0.80291971,  0.19708029]]))
0 accuracy: 0
1 accuracy: 0
Per-class accuracy: 0.555105802551
Accuracy: 0.65799739922
()


In [7]:
rbf = svm.SVC(kernel='poly', degree=1).fit(X_train, y_train)
y_pred = rbf.predict(X_test)
evaluateClassifier(y_test, y_pred)

('Confusion matrix\n', array([[970,  20],
       [501,  47]]))
('Normalized confusion matrix\n', array([[ 0.97979798,  0.02020202],
       [ 0.91423358,  0.08576642]]))
0 accuracy: 0
1 accuracy: 0
Per-class accuracy: 0.532782201578
Accuracy: 0.661248374512


In [8]:
rbf = svm.SVC(kernel='poly', degree=2).fit(X_train, y_train)
y_pred = rbf.predict(X_test)
evaluateClassifier(y_test, y_pred)

('Confusion matrix\n', array([[938,  52],
       [401, 147]]))
('Normalized confusion matrix\n', array([[ 0.94747475,  0.05252525],
       [ 0.73175182,  0.26824818]]))
0 accuracy: 0
1 accuracy: 0
Per-class accuracy: 0.607861461329
Accuracy: 0.705461638492


In [ ]:
rbf = svm.SVC(kernel='poly', degree=3).fit(X_train, y_train)
y_pred = rbf.predict(X_test)
evaluateClassifier(y_test, y_pred)